## Feature Engineering
Engineer interpretable, numeric features for clustering and risk profiling.

## Cleaning Audit
Compare each column before and after cleaning to verify changes are controlled and explainable.

## Data Cleaning Functions
Define and apply robust cleaning logic, including abbreviation detection and case normalization.

## Data Loading
Load the raw dataset and create a backup for auditability.

In [ ]:
# File paths and constants
CURRENT_YEAR = 2026
RANDOM_STATE = 42

# Data paths
ROOT = Path.cwd()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

RAW_DATA_PATH = ROOT / "raw_data" / "champions_group_data.csv"
CLEANED_DATA_PATH = ROOT / "data" / "processed" / "cleaned_base.csv"
FEATURES_RAW_PATH = ROOT / "data" / "processed" / "features_for_clustering_raw.csv"
FEATURES_SCALED_PATH = ROOT / "data" / "processed" / "features_for_clustering_scaled.csv"
FEATURES_SCALED_DROPPED_PATH = ROOT / "data" / "processed" / "features_for_clustering_scaled_dropped.csv"
PCA_DATA_PATH = ROOT / "data" / "processed" / "pca_data.csv"
CLUSTER_PROFILES_PATH = ROOT / "data" / "processed" / "cluster_profiles_no_noise_transposed.csv"
ROW_ID_MAP_PATH = ROOT / "data" / "processed" / "row_id_map.csv"
FEATURE_DICT_PATH = ROOT / "docs" / "feature_dictionary_member2.csv"

# Output paths
OUT_ORIGINAL_WITH_CLUSTERS = ROOT / "data" / "processed" / "original_with_clusters.csv"
OUT_CLUSTER_PROFILE = ROOT / "data" / "processed" / "cluster_summary_original_features.csv"
OUT_PCA_DATA = ROOT / "data" / "processed" / "pca_data.csv"

## Global Configuration
Define all constants, file paths, and random seeds used throughout the notebook.

## Library Imports
All required libraries for data processing, feature engineering, and clustering.

# Champions Group Company Clustering & Risk Profiling

This notebook provides a production-ready, reproducible pipeline for:
- Cleaning and auditing the Champions Group dataset
- Engineering interpretable features for clustering
- Running dimensionality reduction and clustering (PCA, KMeans, HDBSCAN)
- Interpreting and visualizing cluster profiles and risk types

**Authors:** Team SDS Datathon, 2026

---

### Import Packages

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import re
from pathlib import Path
import shutil

# Visualization
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.metrics import mutual_info_score, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import KMeans

# HDBSCAN (for advanced clustering)
import hdbscan

# Scipy (optional, for entropy and sparse matrix check)
from scipy.stats import entropy
try:
    from scipy import sparse
except ImportError:
    sparse = None

### Cell 1 — Load Raw Dataset

This cell loads the original Champions Group dataset into memory.  
We also create a full raw copy (`df_raw_copy`) to ensure:
- Full reproducibility  
- Auditability of all later cleaning steps  
- Ability to compare "before vs after" during cleaning  


In [ ]:
# Load raw dataset and create a backup for audit
df_raw = pd.read_csv(RAW_DATA_PATH)
df_raw_copy = df_raw.copy()
df_raw.shape

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\3972300105.py:2: DtypeWarning: Columns (0: Lattitude, 1: Registration Number Type) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("../raw_data/champions_group_data.csv")


(8559, 72)

### Cell 2 — Define abbreviation detection logic

This cell defines the `is_abbreviation()` helper function used later to preserve uppercase abbreviations during cleaning.


In [ ]:
def is_abbreviation(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.strip()
    if not t:
        return False
    if t.isupper() and len(t) <= 12:
        return True
    uppercase_ratio = sum(c.isupper() for c in t) / max(1, len(t))
    if uppercase_ratio > 0.6 and len(t) <= 18:
        return True
    if re.match(r"^[A-Z0-9][A-Z0-9\-\./]*$", t) and len(t) <= 20:
        return True
    return False

### Cell 3 — Main Data Cleaning Function

This cell defines the core cleaning function used for preprocessing the dataset.  
It handles whitespace cleanup, pseudo-missing value replacement, safe numeric conversion,  
and controlled case normalization with abbreviation protection.


In [ ]:
# Standardize pseudo-missing values and clean data
PSEUDO_MISSING = {"": pd.NA, "na": pd.NA, "n/a": pd.NA, "none": pd.NA, "null": pd.NA, "-": pd.NA}
GEO_COLUMNS = {"country", "country name", "country/region", "region", "state", "state/province", "parent state", "parent state/province", "global ultimate state", "global ultimate state/province", "domestic ultimate state", "domestic ultimate state/province"}
BUSINESS_CATEGORY_COLUMNS = {"entity type", "ownership type", "company status", "company status (active/inactive)", "manufacturing status", "parent company status", "global ultimate company status", "domestic ultimate company status"}

def clean_preserve_abbrev(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    obj_cols = df.select_dtypes(include="object").columns.tolist()
    for col in obj_cols:
        s = df[col].astype("string").str.strip().str.replace(r"\s+", " ", regex=True).replace(PSEUDO_MISSING, regex=False)
        df[col] = s
    for col in obj_cols:
        sample = df[col].dropna().astype(str).head(50)
        numeric_ratio = sample.str.match(r"^-?\d+(\.\d+)?$").mean()
        if numeric_ratio > 0.85:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    case_cols = df.select_dtypes(include=["object", "string"]).columns.tolist()
    for col in case_cols:
        s = df[col]
        col_lc = col.lower().strip()
        if col_lc in GEO_COLUMNS or col_lc in BUSINESS_CATEGORY_COLUMNS:
            df[col] = s.apply(lambda v: v.title() if isinstance(v, str) else v)
            continue
        if col_lc == "franchise status":
            df[col] = s
            continue
        non_null = s.dropna()
        str_values = non_null[non_null.apply(lambda x: isinstance(x, str))]
        if str_values.empty:
            continue
        avg_len = str_values.map(len).mean()
        max_words = (str_values.str.count(" ").max() or 0) + 1
        if avg_len <= 30 and max_words <= 3:
            df[col] = s.apply(lambda v: v if not isinstance(v, str) else (v if is_abbreviation(v) else v.lower()))
    return df

df_clean = clean_preserve_abbrev(df_raw_copy)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\1997451450.py:43: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  obj_cols = df.select_dtypes(include="object").columns.tolist()


### Cell 4 — Cleaning Audit

This cell compares each column before and after cleaning to measure how many
values were changed. It helps verify that the cleaning process is controlled,
minimal, and fully explainable.


In [ ]:
audit_rows = []
for col in df_raw_copy.columns:
    if df_raw_copy[col].dtype == "object":
        before = df_raw_copy[col].astype("string")
        after = df_clean[col].astype("string")
        changed = ((before.fillna("<<NA>>") != after.fillna("<<NA>>")).sum())
        audit_rows.append([col, int(changed), float(changed / len(df_raw_copy))])
audit = pd.DataFrame(audit_rows, columns=["column", "cells_changed_count", "cells_changed_ratio"]).sort_values("cells_changed_ratio", ascending=False)
audit.head(15)

,column,cells_changed_count,cells_changed_ratio
0,Lattitude,4,0.000467
1,Franchise Status,0,0.000000


### Cell 5 — Information Density Score (IDS)

This cell computes the Information Density Score for each column.  
IDS combines value diversity (entropy) and completeness (non-missing ratio)  
to measure how informative each column is for downstream analysis.


In [6]:
# Compute Information Density Score (IDS) for a column
def information_density(col: pd.Series) -> float:

    non_null = col.dropna()

    if non_null.empty or non_null.nunique() <= 1:
        return 0.0
    
    # Probability distribution of values
    probs = non_null.value_counts(normalize=True)

    # Entropy of distribution
    H = entropy(probs)

    # Normalized entropy
    H_norm = H / np.log(len(probs)) if len(probs) > 1 else 0.0
    return float((1 - col.isna().mean()) * H_norm)

# Build summary table for all columns
summary = pd.DataFrame({
    "dtype": df_clean.dtypes.astype(str),
    "missing_ratio": df_clean.isna().mean(),
    "n_unique": df_clean.nunique(dropna=True),
})
summary["IDS"] = df_clean.apply(information_density)

summary.sort_values("IDS", ascending=False).head(10)


,dtype,missing_ratio,n_unique,IDS
DUNS Number,int64,0.000000,8559,1.000000
Company Sites,string,0.000000,8557,0.999990
Parent Company,string,0.000000,7476,0.984701
Domestic Ultimate Company,string,0.000000,7203,0.979771
Global Ultimate Company,string,0.000000,7201,0.979698
Parent Street Address,string,0.012735,7339,0.971538
Address Line 1,string,0.040075,8156,0.959540
Entity Type,str,0.000000,3,0.952594
Domestic Ultimate Street Address,string,0.038790,6986,0.943380
Global Ultimate Street Address,string,0.039958,6967,0.941937


### Cell 6 — Missingness Structure Analysis (Mutual Information)

This cell evaluates whether missing values carry information by measuring
the association between missingness patterns and numeric variables using
Mutual Information (MI).


In [7]:
numeric_cols = df_clean.select_dtypes(include="number").columns.tolist()
missing_structure = {}

for col in df_clean.columns:

    # Missing ratio of the current column
    mr = df_clean[col].isna().mean()

    # Skip columns with low missingness or when no numeric columns exist
    if mr < 0.10 or len(numeric_cols) == 0:
        missing_structure[col] = 0.0
        continue

    # Binary indicator of missingness (1 = missing, 0 = observed)
    miss_flag = df_clean[col].isna().astype(int)
    mi_scores = []

    for nc in numeric_cols:

        # Only consider numeric columns with sufficient non-missing values
        valid = df_clean[nc].notna()
        if valid.sum() <= 50:
            continue
        try:

            # Discretize numeric values into quantile bins
            binned = pd.qcut(df_clean.loc[valid, nc], q=5, duplicates="drop")

             # Compute mutual information between missingness and numeric bins
            mi_scores.append(mutual_info_score(miss_flag.loc[valid], binned))
        except:
            continue
    
    # Use the maximum MI score as the missingness structure indicator
    missing_structure[col] = float(max(mi_scores)) if mi_scores else 0.0

summary["missing_structure_MI"] = pd.Series(missing_structure)
summary.sort_values("missing_structure_MI", ascending=False).head(10)


,dtype,missing_ratio,n_unique,IDS,missing_structure_MI
No. of Routers,str,0.385442,4,0.105415,0.364904
Phone Number,float64,0.804416,1670,0.195559,0.303823
Ownership Type,str,0.126417,5,0.011801,0.248285
No. of Servers,str,0.391284,3,0.005972,0.221281
No. of Storage Devices,str,0.401098,4,0.048510,0.216307
ANZSIC Code,float64,0.833392,136,0.130571,0.201196
ANZSIC Description,string,0.833158,137,0.130736,0.200499
Is Headquarters,float64,0.828601,2,0.097540,0.188650
NACE Rev 2 Description,string,0.823344,177,0.138340,0.179933
ISIC Rev 4 Description,string,0.823110,147,0.137654,0.179612


### Cell 7 — Redundancy Check (Correlation-Based)

This cell identifies redundant numeric features by computing the absolute
correlation matrix and flagging columns with correlation above 0.95.


In [8]:
# Set to store redundant (highly correlated) numeric columns
redundant = set()

if len(numeric_cols) >= 2:

    # Absolute correlation matrix for numeric features
    corr = df_clean[numeric_cols].corr().abs()

    # Identify redundant columns (correlation > 0.95)
    for i in range(len(corr.columns)):
        for j in range(i):
            if corr.iloc[i, j] > 0.95:
                redundant.add(corr.columns[i])
                
# Flag redundant columns in the summary table
summary["redundant"] = summary.index.isin(redundant)
summary[summary["redundant"]].head(20)


,dtype,missing_ratio,n_unique,IDS,missing_structure_MI,redundant
Employees Total,int64,0.000000,247,0.553791,0.000000,True
8-Digit SIC Code,float64,0.620283,445,0.299423,0.065698,True
Ticker,float64,0.999533,4,0.000467,0.002335,True
Longitude,float64,0.776609,326,0.179177,0.146916,True
Parent Postal Code,float64,0.022783,1583,0.779617,0.000000,True
Global Ultimate Postal Code,Int64,0.056665,1563,0.758436,0.000000,True
Domestic Ultimate Postal Code,float64,0.046618,1581,0.766902,0.000000,True
Registration Number,Float64,0.301087,5976,0.698882,0.135618,True
Is Domestic Ultimate,float64,0.823110,2,0.094748,0.179612,True
IT spend,int64,0.000000,1935,0.589199,0.000000,True


### Cell 8 — Final Column Decision Table

This cell generates a final KEEP / DROP / CONDITIONAL decision for each column
based on information density, missingness structure, redundancy, and uniqueness.


In [9]:
# Decision logic for whether to keep or drop each column
def decide(row):
    if row["n_unique"] <= 1:
        return "DROP: constant"

    # Drop columns with extremely high missingness and no information
    if (row["missing_ratio"] > 0.85) and (row["IDS"] < 0.05) and (row["missing_structure_MI"] < 0.01):
        return "DROP: no information"

    # Drop redundant numeric columns
    if bool(row["redundant"]):
        return "DROP: redundant"

    # Keep if missingness is informative (non-random)
    if row["missing_structure_MI"] > 0.02:
        return "KEEP: informative missingness"

    # Keep if information density is high
    if row["IDS"] > 0.15:
        return "KEEP: informative content"

    return "CONDITIONAL: review"

decision_table = summary.reset_index().rename(columns={"index": "column"}).copy()
decision_table["decision"] = summary.apply(decide, axis=1).values
decision_table["decision"].value_counts()


decision
KEEP: informative content        33
KEEP: informative missingness    19
DROP: redundant                   9
CONDITIONAL: review               5
DROP: constant                    5
DROP: no information              1
Name: count, dtype: int64

### Cell 10 — Export Cleaned Data, Audit Report, and Decision Table

This cell provides optional export commands for saving the cleaned dataset,
the cleaning audit log, and the final decision table.  
The lines are commented out so the user can enable them when needed

In [10]:
# # File paths for optional exports
# CLEAN_OUT = "cleaned_base.csv"
# AUDIT_OUT = "cleaning_audit_changes.xlsx"
# DECISION_OUT = "ids_decision_table_no_dictionary.xlsx"

# # Exporting
# df_clean.to_csv(CLEAN_OUT, index=False)
# audit.to_excel(AUDIT_OUT, index=False)
# decision_table.to_excel(DECISION_OUT, index=False)

# Member 2 — Feature Engineering (Signal Construction)

**Goal:** Turn company attributes into **numeric, comparable, explainable signals** for **PCA + clustering**.

**Scope (Member 2):**
- Start from **Member 1's cleaned & imputed dataset** (recommended: `data/processed/clean_base.csv`).
- Do **not** make global cleaning decisions (dropping rows/columns, global imputation strategies, etc.).
- You *may* create **presence indicators** and derived features that treat missingness as signal.

**Outputs (for handoff):**
- `df_features_raw`: numeric feature matrix (not scaled)
- `df_features_scaled`: scaled matrix for PCA/clustering
- `feature_dict`: short description of engineered features

---


In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pathlib import Path
import shutil
import re

pd.set_option('display.max_columns', 250)


In [12]:
# -------------------------------
# Copy clean_base and load
# -------------------------------
ROOT = Path.cwd()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

SOURCE_PATH = ROOT / "data\processed" / "cleaned_base.csv"
DEST_DIR = ROOT / "data" / "processed"
DEST_DIR.mkdir(parents=True, exist_ok=True)

if not SOURCE_PATH.exists():
    raise FileNotFoundError(f"Missing source file: {SOURCE_PATH}")


df = pd.read_csv(SOURCE_PATH)

print("clean_base shape:", df.shape)


clean_base shape: (8559, 72)


<>:8: SyntaxWarning: invalid escape sequence '\p'
<>:8: SyntaxWarning: invalid escape sequence '\p'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\2524596087.py:8: SyntaxWarning: invalid escape sequence '\p'
  SOURCE_PATH = ROOT / "data\processed" / "cleaned_base.csv"
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\2524596087.py:16: DtypeWarning: Columns (0: Registration Number Type) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(SOURCE_PATH)


## 1) Helper functions

We standardize "presence" checks so empty strings don't count as filled.
We also parse range strings like `"1 to 10"` into numeric midpoints.


In [13]:
def has_value(series: pd.Series) -> pd.Series:
    s = series.astype("string").str.strip()
    return s.notna() & s.ne("")

def normalize_text(series: pd.Series) -> pd.Series:
    s = series.astype("string").str.strip().str.upper()
    return s.replace({"": pd.NA})

def range_to_midpoint(value):
    if isinstance(value, str):
        if "to" in value:
            parts = value.split("to")
        elif "-" in value:
            parts = value.split("-")
        else:
            return np.nan
        if len(parts) >= 2:
            try:
                low = float(parts[0].strip())
                high = float(parts[1].strip())
                return (low + high) / 2
            except ValueError:
                return np.nan
    return np.nan

_split_re = re.compile(r"[;,|]+")

def count_delimited_items(value) -> int:
    if value is None:
        return 0
    if not isinstance(value, str):
        value = str(value)
    s = value.strip()
    if s == "":
        return 0
    parts = [p.strip() for p in _split_re.split(s) if p.strip()]
    return len(parts) if parts else 0

def safe_to_numeric(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

def add_numeric_feature(features: pd.DataFrame, series: pd.Series, name: str, log: bool = True):
    series = safe_to_numeric(series)
    missing = series.isna()
    features[f"{name}_missing"] = missing.astype(int)
    if series.notna().any():
        fill_value = series.median()
        series_filled = series.fillna(fill_value)
    else:
        series_filled = series.fillna(0)
    features[name] = series_filled
    if log:
        features[f"log_{name}"] = np.log1p(series_filled.clip(lower=0))

def missing_ratio_for(cols):
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return pd.Series(0, index=df.index)
    present = {}
    for c in cols:
        s = df[c]
        if pd.api.types.is_numeric_dtype(s):
            present[c] = s.notna()
        else:
            present[c] = has_value(s)
    present_df = pd.DataFrame(present)
    return 1 - present_df.mean(axis=1)


## 2) Core transparency + structure signals (curated short names)

These are your **interpretable, low-risk** binary signals.


In [14]:
features = pd.DataFrame(index=df.index)

def add_has(col, feature_name):
    if col in df.columns:
        features[feature_name] = has_value(df[col]).astype(int)

# Transparency / traceability
add_has("Website", "has_website")
add_has("Phone Number", "has_phone")
add_has("Address Line 1", "has_address")
add_has("City", "has_city")
add_has("State", "has_state")
add_has("State Or Province Abbreviation", "has_state_abbrev")
add_has("Postal Code", "has_postal_code")
add_has("Country", "has_country")
add_has("Region", "has_region")

# Company name (column appears labeled as Company Sites in this dataset)
add_has("Company Sites", "has_company_name")

# Ownership / structure
add_has("Parent Company", "has_parent")
add_has("Global Ultimate Company", "has_global_ultimate")
add_has("Domestic Ultimate Company", "has_domestic_ultimate")

# Verifiability extras
add_has("Ticker", "has_ticker")
add_has("Registration Number", "has_registration_number")
add_has("Company Description", "has_company_description")
add_has("Legal Status", "has_legal_status")
add_has("Ownership Type", "has_ownership_type")
add_has("Entity Type", "has_entity_type")

# Company status (value-coded)
status_col = "Company Status (Active/Inactive)"
if status_col in df.columns:
    status = df[status_col].astype("string").str.strip().str.lower()
    status_map = {"active": 1, "inactive": 0}
    features["company_status_binary"] = status.map(status_map)
    features["has_company_status"] = features["company_status_binary"].notna().astype(int)
    features["company_status_binary"] = features["company_status_binary"].fillna(0)


## 3) Credibility / completeness score (and missing_ratio)

We include **status-known** (not status value) so inactive firms aren't penalized.


In [15]:
credibility_flag_cols = [c for c in [
    "has_website", "has_address", "has_phone",
    "has_ticker", "has_parent", "has_global_ultimate", "has_domestic_ultimate",
    "has_registration_number", "has_company_description",
    "has_company_status"
] if c in features.columns]

if credibility_flag_cols:
    features["credibility_score"] = features[credibility_flag_cols].sum(axis=1)
    features["credibility_score_norm"] = features["credibility_score"] / len(credibility_flag_cols)
    features["missing_ratio_credibility"] = 1 - features["credibility_score_norm"]

# Missingness ratios by group
contact_cols = [
    "Website", "Phone Number", "Address Line 1", "City", "State",
    "State Or Province Abbreviation", "Postal Code", "Country", "Region"
]
ownership_cols = [
    "Parent Company", "Parent Country/Region",
    "Global Ultimate Company", "Global Ultimate Country Name",
    "Domestic Ultimate Company"
]
financial_cols = [
    "Employees Single Site", "Employees Total", "Revenue (USD)", "Market Value (USD)",
    "Corporate Family Members", "Year Found"
]
it_cols = [
    "No. of PC", "No. of Desktops", "No. of Laptops", "No. of Routers",
    "No. of Servers", "No. of Storage Devices", "IT Budget", "IT spend"
]
code_cols = [
    "SIC Code", "8-Digit SIC Code", "NAICS Code", "NACE Rev 2 Code",
    "ANZSIC Code", "ISIC Rev 4 Code"
]

features["missing_ratio_contact"] = missing_ratio_for(contact_cols)
features["missing_ratio_ownership"] = missing_ratio_for(ownership_cols)
features["missing_ratio_financial"] = missing_ratio_for(financial_cols)
features["missing_ratio_it"] = missing_ratio_for(it_cols)
features["missing_ratio_codes"] = missing_ratio_for(code_cols)

all_missing_cols = contact_cols + ownership_cols + financial_cols + it_cols + code_cols
features["missing_ratio_overall"] = missing_ratio_for(all_missing_cols)


## 4) Organisational complexity (group size by global ultimate)

Companies sharing the same global ultimate are treated as belonging to the same group.


In [16]:
if "Global Ultimate Company" in df.columns:
    key = normalize_text(df["Global Ultimate Company"])
    group_sizes = key.groupby(key).transform("size")
    features["org_complexity_count"] = group_sizes.fillna(0).astype(int)
    features["log_org_complexity_count"] = np.log1p(features["org_complexity_count"])


## 5) Scale + market signals

These control for size so clustering isn't just "big vs small".


In [17]:
# Coerce numeric fields and add log versions
numeric_cols = {
    "Employees Total": "employees_total",
    "Employees Single Site": "employees_single_site",
    "Revenue (USD)": "revenue_usd",
    "Market Value (USD)": "market_value_usd",
    "Corporate Family Members": "corporate_family_members",
}

for col, name in numeric_cols.items():
    if col in df.columns:
        add_numeric_feature(features, df[col], name, log=True)

# Company age
CURRENT_YEAR = 2026
if "Year Found" in df.columns:
    year_found = safe_to_numeric(df["Year Found"])
    company_age = CURRENT_YEAR - year_found
    company_age = company_age.where((company_age >= 0) & (company_age <= 300))
    features["company_age_missing"] = company_age.isna().astype(int)
    fill_value = company_age.median()
    if pd.isna(fill_value):
        fill_value = 0
    company_age_filled = company_age.fillna(fill_value)
    features["company_age"] = company_age_filled
    features["log_company_age"] = np.log1p(company_age_filled.clip(lower=0))

# Ratios
if "employees_total" in features.columns and "employees_single_site" in features.columns:
    denom = features["employees_total"].replace(0, np.nan)
    features["employee_concentration"] = (features["employees_single_site"] / denom).fillna(0)

if "revenue_usd" in features.columns and "employees_total" in features.columns:
    denom = features["employees_total"].replace(0, np.nan)
    features["revenue_per_employee"] = (features["revenue_usd"] / denom).fillna(0)

if "market_value_usd" in features.columns and "employees_total" in features.columns:
    denom = features["employees_total"].replace(0, np.nan)
    features["market_value_per_employee"] = (features["market_value_usd"] / denom).fillna(0)


## 6) Geography + multinational heuristics

We avoid one-hot encoding high-cardinality city names. Use country/region + parent/ultimate country comparisons.


In [18]:
# Coordinates
if "Lattitude" in df.columns:
    add_numeric_feature(features, df["Lattitude"], "latitude", log=False)

if "Longitude" in df.columns:
    add_numeric_feature(features, df["Longitude"], "longitude", log=False)

entity_country = normalize_text(df["Country"]) if "Country" in df.columns else pd.Series(pd.NA, index=df.index)
parent_country = normalize_text(df["Parent Country/Region"]) if "Parent Country/Region" in df.columns else pd.Series(pd.NA, index=df.index)
global_country = normalize_text(df["Global Ultimate Country Name"]) if "Global Ultimate Country Name" in df.columns else pd.Series(pd.NA, index=df.index)

parent_present = has_value(df["Parent Company"]) if "Parent Company" in df.columns else pd.Series(False, index=df.index)
global_present = has_value(df["Global Ultimate Company"]) if "Global Ultimate Company" in df.columns else pd.Series(False, index=df.index)

features["parent_foreign_flag"] = (parent_present & parent_country.notna() & (parent_country != entity_country)).astype(int)
features["global_ultimate_foreign_flag"] = (global_present & global_country.notna() & (global_country != entity_country)).astype(int)
features["multinational_flag"] = ((features["parent_foreign_flag"] == 1) | (features["global_ultimate_foreign_flag"] == 1)).astype(int)

countries_df = pd.concat([entity_country, parent_country, global_country], axis=1)
features["num_countries_reported"] = countries_df.nunique(axis=1, dropna=True)


## 7) IT / operational footprint signals

We parse IT asset ranges into midpoints, then build intensity & composition signals.


In [19]:
def parse_range_or_numeric(value):
    if isinstance(value, (int, float)) and not pd.isna(value):
        return float(value)
    if isinstance(value, str):
        mid = range_to_midpoint(value)
        if not pd.isna(mid):
            return mid
        try:
            return float(value.strip())
        except ValueError:
            return np.nan
    return np.nan

asset_cols_map = {
    "No. of PC": "pc_midpoint",
    "No. of Desktops": "desktops_midpoint",
    "No. of Laptops": "laptops_midpoint",
    "No. of Routers": "routers_midpoint",
    "No. of Servers": "servers_midpoint",
    "No. of Storage Devices": "storage_devices_midpoint",
}

for col, name in asset_cols_map.items():
    if col in df.columns:
        series = df[col].apply(parse_range_or_numeric)
        add_numeric_feature(features, series, name, log=True)

asset_feature_cols = [name for name in asset_cols_map.values() if name in features.columns]
if asset_feature_cols:
    features["it_assets_total"] = features[asset_feature_cols].sum(axis=1)
    features["log_it_assets_total"] = np.log1p(features["it_assets_total"])

if "IT Budget" in df.columns:
    add_numeric_feature(features, df["IT Budget"], "it_budget", log=True)
if "IT spend" in df.columns:
    add_numeric_feature(features, df["IT spend"], "it_spend", log=True)

if "it_budget" in features.columns and "it_spend" in features.columns:
    denom = features["it_budget"].replace(0, np.nan)
    features["it_spend_rate"] = (features["it_spend"] / denom).fillna(0).clip(lower=0, upper=3)
    features["it_budget_gap"] = features["it_budget"] - features["it_spend"]
    features["log_abs_it_budget_gap"] = np.log1p(features["it_budget_gap"].abs())

if "it_assets_total" in features.columns and "employees_total" in features.columns:
    denom = features["employees_total"].replace(0, np.nan)
    features["it_assets_per_employee"] = (features["it_assets_total"] / denom).fillna(0)

if "it_spend" in features.columns and "employees_total" in features.columns:
    denom = features["employees_total"].replace(0, np.nan)
    features["it_spend_per_employee"] = (features["it_spend"] / denom).fillna(0)


## 8) Industry code features (low-cardinality sector buckets)

Full industry codes can be high-cardinality. For clustering/PCA, we use **2-digit buckets** (industry sectors).


In [20]:
code_cols = {
    "SIC Code": "sic_code",
    "8-Digit SIC Code": "sic8_code",
    "NAICS Code": "naics_code",
    "NACE Rev 2 Code": "nace2_code",
    "ANZSIC Code": "anzsic_code",
    "ISIC Rev 4 Code": "isic4_code",
}

for col, prefix in code_cols.items():
    if col in df.columns:
        counts = df[col].astype("string").apply(count_delimited_items)
        features[f"{prefix}_count"] = counts
        features[f"has_{prefix}"] = (counts > 0).astype(int)

if "Registration Number" in df.columns:
    reg_counts = df["Registration Number"].astype("string").apply(count_delimited_items)
    features["registration_number_count"] = reg_counts


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\650323838.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features["registration_number_count"] = reg_counts


## 9) Categorical encoding (PCA/clustering ready)

We one-hot encode selected low-cardinality categoricals + sector buckets.


In [21]:
bool_map = {"true": 1, "false": 0, "yes": 1, "no": 0, "y": 1, "n": 0, "1": 1, "0": 0}

def add_boolean_feature(col, name):
    if col not in df.columns:
        return
    s = df[col].astype("string").str.strip().str.lower()
    mapped = s.map(bool_map)
    features[name] = mapped.fillna(0).astype(int)
    features[f"{name}_missing"] = mapped.isna().astype(int)

add_boolean_feature("Is Headquarters", "is_headquarters")
add_boolean_feature("Is Domestic Ultimate", "is_domestic_ultimate")

candidate_categoricals = [
    "Region",
    "Entity Type",
    "Ownership Type",
    "Legal Status",
    "Franchise Status",
    "Manufacturing Status",
    "Registration Number Type",
]

categorical_cols = []
for col in candidate_categoricals:
    if col in df.columns and df[col].nunique(dropna=True) <= 20:
        categorical_cols.append(col)

if categorical_cols:
    df_cats = df[categorical_cols].copy()
    for col in categorical_cols:
        df_cats[col] = df_cats[col].astype("string").str.strip()
    df_dummies = pd.get_dummies(df_cats, prefix=categorical_cols, prefix_sep="__", dummy_na=True)
else:
    df_dummies = pd.DataFrame(index=df.index)

print("Categorical columns encoded:", categorical_cols)
print("One-hot shape:", df_dummies.shape)


Categorical columns encoded: ['Region', 'Entity Type', 'Ownership Type', 'Legal Status', 'Franchise Status', 'Manufacturing Status', 'Registration Number Type']
One-hot shape: (8559, 31)


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\3519656746.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[name] = mapped.fillna(0).astype(int)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\3519656746.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[f"{name}_missing"] = mapped.isna().astype(int)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\3519656746.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

## 10) Build final feature matrices (raw + scaled)

- Drop obvious identifiers & free-text
- Keep numeric + engineered features + one-hot columns
- Create scaled matrix for PCA/clustering


In [22]:
df_features_raw = pd.concat([features, df_dummies], axis=1)

# Drop columns that are fully missing
df_features_raw = df_features_raw.dropna(axis=1, how="all")

# Ensure numeric-only
df_features_raw = df_features_raw.apply(pd.to_numeric, errors="coerce")

print("Feature matrix shape (pre-impute):", df_features_raw.shape)


Feature matrix shape (pre-impute): (8559, 140)


In [23]:
# Fix any remaining NaNs
if df_features_raw.isna().any().any():
    nan_cols = df_features_raw.columns[df_features_raw.isna().any()].tolist()
    print("NaNs found in raw features. Top columns:", nan_cols[:20])
    for col in nan_cols:
        series = df_features_raw[col]
        fill_value = series.median()
        if pd.isna(fill_value):
            fill_value = 0
        df_features_raw[col] = series.fillna(fill_value)

assert not df_features_raw.isna().any().any(), "NaNs remain in raw features"

# Scale for PCA/clustering
scaler = StandardScaler()
df_features_scaled = pd.DataFrame(
    scaler.fit_transform(df_features_raw),
    columns=df_features_raw.columns,
    index=df_features_raw.index
)

assert not df_features_scaled.isna().any().any(), "NaNs remain in scaled features"


In [24]:
def find_first_column(candidates):
    for col in candidates:
        if col in df.columns:
            return col
    return None

duns_col = None
for col in df.columns:
    if col.strip().lower() == "duns number":
        duns_col = col
        break

name_col = find_first_column(["Company Name", "Company", "Company Sites"])

row_id_map = pd.DataFrame({"row_index": df.index})
if duns_col:
    row_id_map["DUNS Number"] = df[duns_col]
if name_col:
    row_id_map["Company Name"] = df[name_col]

print("Row id map columns:", row_id_map.columns.tolist())


Row id map columns: ['row_index', 'DUNS Number', 'Company Name']


In [25]:
feature_descriptions = {
    "has_website": "1 if Website is present",
    "has_phone": "1 if Phone Number is present",
    "has_address": "1 if Address Line 1 is present",
    "has_city": "1 if City is present",
    "has_state": "1 if State is present",
    "has_state_abbrev": "1 if State Or Province Abbreviation is present",
    "has_postal_code": "1 if Postal Code is present",
    "has_country": "1 if Country is present",
    "has_region": "1 if Region is present",
    "has_company_name": "1 if Company Sites (company name) is present",
    "has_parent": "1 if Parent Company is present",
    "has_global_ultimate": "1 if Global Ultimate Company is present",
    "has_domestic_ultimate": "1 if Domestic Ultimate Company is present",
    "has_ticker": "1 if Ticker is present",
    "has_registration_number": "1 if Registration Number is present",
    "has_company_description": "1 if Company Description is present",
    "has_legal_status": "1 if Legal Status is present",
    "has_ownership_type": "1 if Ownership Type is present",
    "has_entity_type": "1 if Entity Type is present",
    "company_status_binary": "1 if Company Status is active, 0 if inactive or missing",
    "has_company_status": "1 if Company Status is present",
    "credibility_score": "Count of core transparency/ownership flags present",
    "credibility_score_norm": "Credibility score normalized by number of flags",
    "missing_ratio_credibility": "1 - credibility_score_norm",
    "missing_ratio_contact": "Missing ratio across contact fields",
    "missing_ratio_ownership": "Missing ratio across ownership fields",
    "missing_ratio_financial": "Missing ratio across financial fields",
    "missing_ratio_it": "Missing ratio across IT fields",
    "missing_ratio_codes": "Missing ratio across industry code fields",
    "missing_ratio_overall": "Missing ratio across key groups",
    "org_complexity_count": "Count of entities sharing the same global ultimate",
    "log_org_complexity_count": "log1p of org_complexity_count",
    "employees_total": "Employees total (imputed)",
    "employees_single_site": "Employees single site (imputed)",
    "revenue_usd": "Revenue USD (imputed)",
    "market_value_usd": "Market value USD (imputed)",
    "corporate_family_members": "Corporate family members (imputed)",
    "company_age": "Company age in years (imputed)",
    "employee_concentration": "Employees single site divided by employees total",
    "revenue_per_employee": "Revenue per employee (USD)",
    "market_value_per_employee": "Market value per employee (USD)",
    "latitude": "Latitude (imputed)",
    "longitude": "Longitude (imputed)",
    "parent_foreign_flag": "1 if parent country differs from entity country",
    "global_ultimate_foreign_flag": "1 if global ultimate country differs from entity country",
    "multinational_flag": "1 if parent or global ultimate is foreign",
    "num_countries_reported": "Count of unique countries reported (entity/parent/global)",
    "pc_midpoint": "Midpoint of PC count range (imputed)",
    "desktops_midpoint": "Midpoint of desktop count range (imputed)",
    "laptops_midpoint": "Midpoint of laptop count range (imputed)",
    "routers_midpoint": "Midpoint of router count range (imputed)",
    "servers_midpoint": "Midpoint of server count range (imputed)",
    "storage_devices_midpoint": "Midpoint of storage device count range (imputed)",
    "it_assets_total": "Sum of IT asset midpoints",
    "log_it_assets_total": "log1p of it_assets_total",
    "it_budget": "IT budget (imputed)",
    "it_spend": "IT spend (imputed)",
    "it_spend_rate": "IT spend divided by IT budget (clipped)",
    "it_budget_gap": "IT budget minus IT spend",
    "log_abs_it_budget_gap": "log1p of absolute IT budget gap",
    "it_assets_per_employee": "IT assets per employee",
    "it_spend_per_employee": "IT spend per employee",
    "sic_code_count": "Number of SIC codes (split on delimiters)",
    "sic8_code_count": "Number of 8-digit SIC codes (split on delimiters)",
    "naics_code_count": "Number of NAICS codes (split on delimiters)",
    "nace2_code_count": "Number of NACE Rev 2 codes (split on delimiters)",
    "anzsic_code_count": "Number of ANZSIC codes (split on delimiters)",
    "isic4_code_count": "Number of ISIC Rev 4 codes (split on delimiters)",
    "has_sic_code": "1 if SIC Code is present",
    "has_sic8_code": "1 if 8-digit SIC Code is present",
    "has_naics_code": "1 if NAICS Code is present",
    "has_nace2_code": "1 if NACE Rev 2 Code is present",
    "has_anzsic_code": "1 if ANZSIC Code is present",
    "has_isic4_code": "1 if ISIC Rev 4 Code is present",
    "registration_number_count": "Number of registration numbers (split on delimiters)",
}

def describe_feature(name):
    if name in feature_descriptions:
        return feature_descriptions[name]
    if name.endswith("_missing"):
        base = name[:-8]
        return f"Missing indicator for {base}"
    if name.startswith("log_"):
        base = name[4:]
        return f"log1p of {base}"
    if "__" in name:
        base, val = name.split("__", 1)
        val_norm = val.strip().lower()
        if val_norm in ("nan", "<na>"):
            return f"Missing indicator for {base}"
        return f"One-hot: {base} = {val}"
    return "Derived numeric feature"

feature_dict = pd.DataFrame({
    "feature": df_features_raw.columns,
    "description": [describe_feature(c) for c in df_features_raw.columns],
})


In [26]:
# Drop Similar columns
cols_to_drop = [
    "has_state_abbrev",
    "company_status_binary",
    "credibility_score",
    "missing_ratio_credibility",
    "org_complexity_count",
    "employees_total",
    "log_employees_single_site",
    "revenue_usd",
    "market_value_usd",
    "corporate_family_members",
    "company_age",
    "pc_midpoint",
    "desktops_midpoint",
    "laptops_midpoint",
    "routers_midpoint",
    "servers_midpoint",
    "storage_devices_midpoint",
    "it_assets_total",
    "it_budget",
    "it_spend",
    "it_budget_gap",]

bool_list = [col in df_features_scaled.columns for col in cols_to_drop]
print(bool_list)
df_features_scaled_drop = df_features_scaled.drop(columns=cols_to_drop)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


## 11) Export for Member 3 (optional)

Uncomment to export once Member 1's clean_base.csv is ready.


In [27]:
# OUTPUT_DIR = DEST_DIR
# DOCS_DIR = ROOT / "docs"
# DOCS_DIR.mkdir(parents=True, exist_ok=True)

# features_raw_path = OUTPUT_DIR / "features_for_clustering_raw.csv"
# features_scaled_path = OUTPUT_DIR / "features_for_clustering_scaled.csv"
# row_id_map_path = OUTPUT_DIR / "row_id_map.csv"
# feature_dict_path = DOCS_DIR / "feature_dictionary_member2.csv"

# df_features_raw.to_csv(features_raw_path, index=False)
# df_features_scaled.to_csv(features_scaled_path, index=False)
# row_id_map.to_csv(row_id_map_path, index=False)
# feature_dict.to_csv(feature_dict_path, index=False)

# print("Saved:", features_raw_path)
# print("Saved:", features_scaled_path)
# print("Saved:", row_id_map_path)
# print("Saved:", feature_dict_path)

# print("clean_base shape:", df.shape)
# print("Raw features shape:", df_features_raw.shape)
# print("Scaled features shape:", df_features_scaled.shape)
# print("Any NaNs in raw features?", df_features_raw.isna().any().any())
# print("Any NaNs in scaled features?", df_features_scaled.isna().any().any())
# print("Number of feature columns:", df_features_raw.shape[1])

# assert df_features_raw.shape[0] == df.shape[0]
# assert df_features_scaled.shape[0] == df.shape[0]

df_features_scaled_drop.to_csv(
    "features_for_clustering_scaled_dropped.csv",
    index=False
)


# PCA + KMeans Clustering (Reproducible Pipeline)

This notebook clusters companies using a pre-built, **scaled feature matrix** (for modeling) and links the resulting cluster labels back to a more readable **original dataset** (for interpretation). It produces two outputs: (1) the original dataset with a `cluster` column, and (2) a cluster profile summary computed on original features.

In [28]:
# B) Imports (consolidated)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import silhouette_score

In [29]:
# C) Config / File Paths (centralized)
SCALED_PATH = "../data/processed/features_for_clustering_scaled_dropped.csv"
ORIGINAL_PATH = "../data/processed/cleaned_base.csv"

OUT_ORIGINAL_WITH_CLUSTERS = "../data/processed/original_with_clusters.csv"
OUT_CLUSTER_PROFILE = "../data/processed/cluster_summary_original_features.csv"

# PCA-transformed (reduced) feature space output
OUT_PCA_DATA = "../data/processed/pca_data.csv"

RANDOM_STATE = 42

In [30]:
# D) Load Data (scaled features + original readable dataset)
df_scaled = pd.read_csv(SCALED_PATH)
df_original = pd.read_csv(ORIGINAL_PATH, low_memory=False)

print("df_scaled shape   :", df_scaled.shape)
print("df_original shape :", df_original.shape)

# Validate row alignment (must be same row order for label linking)
if len(df_scaled) != len(df_original):
    raise ValueError(
        f"Row mismatch! df_scaled has {len(df_scaled)} rows but df_original has {len(df_original)} rows.\n"
        "They MUST represent the same entities in the same order so cluster labels can be linked back safely.\n"
        "Fix upstream preprocessing to avoid dropping/shuffling rows, or re-export the aligned files."
    )

# Optional stronger check if a stable row id exists in both files
candidate_id_cols = ["row_id", "RowID", "id", "ID"]
id_col = next((c for c in candidate_id_cols if c in df_scaled.columns and c in df_original.columns), None)
if id_col is not None:
    if not (df_scaled[id_col].astype(str).values == df_original[id_col].astype(str).values).all():
        raise ValueError(
            f"Row order mismatch detected via column '{id_col}'.\n"
            "The two files must be aligned row-by-row before clustering labels can be joined back."
        )

df_scaled shape   : (8559, 119)
df_original shape : (8559, 72)


## Dimensionality Reduction

We reduce dimensionality to make KMeans faster and more stable.
- If the feature matrix is **dense** (like a numeric CSV), we use **PCA** with `n_components=0.95` to keep 95% variance.
- If the feature matrix is **sparse** (common after one-hot encoding), PCA may require centering and can become expensive; in that case we use **TruncatedSVD**, which works directly with sparse matrices without densifying.

In [31]:
# E2) Compute PCA (dense) or TruncatedSVD (sparse) -> df_pca
X = df_scaled

# Detect sparse matrices without forcing a SciPy dependency (SciPy is optional here)
is_sparse = False
try:
    from scipy import sparse  # optional; only used for sparse checks
    is_sparse = sparse.issparse(X)
except Exception:
    is_sparse = False

if is_sparse:
    # TruncatedSVD does not support n_components as a variance ratio directly.
    # Use a reasonable cap; increase if cumulative variance is too low for your use case.
    n_components = min(200, X.shape[1] - 1) if X.shape[1] > 1 else 1
    reducer = TruncatedSVD(n_components=n_components, random_state=RANDOM_STATE)
    X_reduced = reducer.fit_transform(X)
    explained = float(np.sum(reducer.explained_variance_ratio_))
    reducer_name = "TruncatedSVD"
else:
    reducer = PCA(n_components=0.95, random_state=RANDOM_STATE)
    X_reduced = reducer.fit_transform(X)
    explained = float(np.sum(reducer.explained_variance_ratio_))
    reducer_name = "PCA"

df_pca = pd.DataFrame(X_reduced, columns=[f"PC{i+1}" for i in range(X_reduced.shape[1])])

# Save reduced feature space for downstream modeling/analysis
df_pca.to_csv(OUT_PCA_DATA, index=False)
print("Saved PCA-transformed data to:", OUT_PCA_DATA)

print(f"Reducer: {reducer_name}")
print("Original shape:", X.shape)
print("Reduced shape :", df_pca.shape)
print("Explained variance (sum):", explained)

df_pca.head()

Saved PCA-transformed data to: ../data/processed/pca_data.csv
Reducer: PCA
Original shape: (8559, 119)
Reduced shape : (8559, 38)
Explained variance (sum): 0.9525713624480342


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38
0,-4.537217,0.630544,-0.217801,-0.817862,-2.161057,-0.297641,1.370103,0.221009,1.386415,-0.559805,-0.161413,1.495649,-1.090911,-0.054848,-0.861624,-0.629427,0.453851,2.092562,-0.045079,-0.483324,1.266398,-0.217258,0.731067,-0.248193,-0.415900,0.127489,0.157482,0.140437,-0.069049,0.119451,0.353414,0.181707,-0.069740,0.060509,0.369749,-0.623337,0.012027,-0.215178
1,2.291527,1.079836,-1.216945,0.136597,-0.354121,0.031126,-0.017264,-0.499659,-0.676079,0.156081,0.828052,-0.349787,0.258328,-0.103964,0.684215,0.762840,-0.442746,1.124972,0.484683,0.339745,-0.503690,0.237295,-0.854476,0.245403,-0.402466,0.092020,-0.350605,-0.011454,0.072727,0.113641,1.837569,-0.271617,-0.247642,-0.999004,0.120161,-0.314665,-0.039607,0.135777
2,3.022971,3.553970,1.231341,-1.883836,2.032231,-1.309641,0.184336,-2.022148,1.400484,-0.436309,-1.342077,2.051998,-0.273623,-0.087288,-0.768106,1.389976,-4.925855,-1.691978,0.767850,1.944092,0.875667,-0.549927,0.750683,0.695698,-0.253858,0.368387,-0.560828,0.001774,1.186508,-1.418292,-1.879591,-0.857522,0.012763,-1.257616,-0.893885,-0.029517,0.024819,0.092893
3,-0.890908,-1.003383,1.180976,6.423441,5.959999,-0.269068,2.219326,0.703031,0.016261,-0.191530,-0.445367,-0.882688,-0.391520,0.320621,-0.810012,1.174865,1.483704,1.667939,-0.060858,-0.229643,-0.796302,-0.276149,-1.696303,0.894115,-0.018737,-0.058796,-0.234769,-0.018936,-0.321411,0.087868,-0.741366,-0.028584,-0.131432,0.987870,-0.016218,0.643174,0.076683,-0.175362
4,2.703361,1.270091,-1.693008,0.101863,0.425518,-0.376443,-1.077080,1.368058,-0.829252,-0.062024,1.048836,0.121653,-0.426292,-0.347429,0.566522,-0.191670,-0.338054,2.179105,0.462040,-0.262363,0.823999,-0.020187,-1.246083,0.545022,-0.479731,0.139223,-0.341949,0.201512,-0.492371,-0.091078,0.294507,0.213689,0.028330,0.323540,0.371315,0.128086,0.123632,0.061353


## Member 3: Clustering & Risk Interpretion
This dataset does not contain outcome labels such as defaults, losses, fraud, or failures.
Therefore, risk is not predictive and not financial risk in the strict sense.

In this analysis, risk is defined as elevated uncertainty arising from a company’s:

	- operating profile
	- organisational structure
	- information transparency

Risk reflects how difficult a company is to assess, monitor, or compare, rather than whether it is “bad”.

# Types of risk that are in scope

Based on the available variables, this dataset supports structural and operational risk analysis, specifically:

**1. Transparency & disclosure risk**

These capture how much information a company makes available and how easy it is to assess.

	- Missing or incomplete public-facing information
	- Lack of financial or size indicators
	- Minimal descriptive detail about operations

**2. Operational complexity risk**

These capture the scale and coordination burden of a company’s operations.

	- Large number of operating sites
	- Large workforce size
	- Broad geographic footprint
	- Extensive IT infrastructure

**3. Organisational & ownership complexity risk**

These capture how layered or decentralised control and accountability may be.

	- Presence of parent entities
	- Global or domestic ultimate owners
	- Large corporate family structures
	- Franchise or decentralised operating models

**4. Data quality risk**

These capture the reliability and completeness of reported information.

	- Missing or inconsistent addresses
	- Missing registration or legal identifiers
	- Incomplete geographic information
	- Conflicting records across fields


In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import hdbscan

## 1.Load features


In [33]:
df_pca = pd.read_csv("../data/processed/pca_data.csv")
X = df_pca.select_dtypes(include="number").to_numpy()

df_feature = pd.read_csv("../data/processed/features_for_clustering_scaled_dropped.csv")
Y = df_feature

print(df_pca.shape)
assert len(df_pca) == len(df_feature), "Row count mismatch: PCA rows and feature rows not aligned"



(8559, 38)


## 2. Clustering (HDBSCAN) and Interpretation

In [34]:
## Run HDBSCAN ##
import os

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=50,
    min_samples=25
)

labels = clusterer.fit_predict(X)

pd.Series(labels).value_counts()

df_clusters = Y.copy()
df_clusters["cluster"] = labels
cluster_profiles = df_clusters.groupby("cluster").mean()

###############
# Separate Real Clusters VS Noise
cluster_profiles_no_noise = cluster_profiles.loc[cluster_profiles.index != -1]

# Save cluster profile tables
out_dir = "../data/processed"
os.makedirs(out_dir, exist_ok=True)


cluster_profiles_no_noise.T.to_csv(
    os.path.join(out_dir, "cluster_profiles_no_noise_transposed.csv"),
    index=True,
)

cluster_profiles_no_noise.T


cluster,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
has_website,0.014992,0.310653,0.288752,1.353961,-0.262791,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,3.562920,3.562920,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669,-0.280669
has_phone,0.088707,-0.105225,0.767470,1.158678,-0.411007,-0.493089,-0.493089,-0.183478,-0.493089,-0.493089,-0.493089,-0.493089,-0.493089,-0.493089,-0.493089,-0.458074,-0.493089,-0.493089,-0.493089,-0.493089,-0.493089,1.166778,-0.394608,-0.493089,-0.493089,-0.493089,0.515358,-0.485674,2.028029,-0.493089,-0.454303,-0.454303,-0.455461,-0.463429,0.722450,-0.493089,-0.493089,0.255758,-0.493089,-0.493089,-0.493089,-0.493089,2.028029,2.028029,-0.493089,-0.493089
has_address,-0.040799,0.204323,0.204323,0.204323,0.204323,-4.814552,-4.256899,0.204323,0.204323,0.204323,-4.760044,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323,0.204323
has_city,0.009939,0.188012,0.252766,0.252766,0.233189,-3.956229,-3.956229,0.252766,0.252766,0.252766,-3.956229,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766,0.252766
has_state,0.010422,0.188333,0.253027,0.253027,0.233468,-3.952141,-3.952141,0.253027,0.253027,0.253027,-3.952141,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027,0.253027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Franchise Status__<NA>,-0.197596,-1.547605,0.199282,-0.677664,0.411478,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,-1.547605,-1.547605,-1.547605,-1.547605,-1.547605,-0.784556,-1.221968,-1.547605,-1.547605,-1.547605,-1.547605,0.646160,0.646160,0.646160,-0.535098,0.646160,0.646160,-0.644290,-0.267909,-1.547605,0.620043,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160,0.646160
Manufacturing Status__Yes,-0.009585,0.001599,0.145754,5.638537,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351,-0.177351
Manufacturing Status__<NA>,0.009585,-0.001599,-0.145754,-5.638537,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351,0.177351
Registration Number Type__indonesia legalization number,-0.030587,-0.030587,-0.030587

In [35]:
# Attach cluster labels to raw champions group data
raw_path = "../raw_data/champions_group_data.csv"
df_raw = pd.read_csv(raw_path)

if len(df_raw) != len(labels):
    raise ValueError(
        f"Row count mismatch: raw_data has {len(df_raw)} rows, labels has {len(labels)} rows"
    )

df_raw_with_cluster = df_raw.copy()
df_raw_with_cluster["cluster_id"] = labels

out_path = "../raw_data/champions_group_data_with_cluster.csv"
df_raw_with_cluster.to_csv(out_path, index=False)
print(f"Saved clustered data to: {out_path}")


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_26152\6041050.py:3: DtypeWarning: Columns (0: Lattitude, 1: Registration Number Type) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(raw_path)


Saved clustered data to: ../raw_data/champions_group_data_with_cluster.csv


In [36]:
n_companies, n_features = X.shape
n_noise = (labels == -1).sum()
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

n_companies, n_features, n_clusters, n_noise, n_noise / n_companies

(8559, 38, 46, np.int64(2069), np.float64(0.24173384741208084))

In [37]:
# First 5 rows for each cluster (set include_noise=False to skip cluster == -1)
from IPython.display import display

include_noise = True

clusters = sorted(pd.unique(df_clusters["cluster"]))
if not include_noise:
    clusters = [c for c in clusters if c != -1]

for c in clusters:
    print(f"\n=== Cluster {c} (n={(df_clusters['cluster'] == c).sum()}) ===")
    display(df_clusters.loc[df_clusters["cluster"] == c].head(5))



=== Cluster -1 (n=2069) ===


,has_website,has_phone,has_address,has_city,has_state,has_postal_code,has_country,has_region,has_company_name,has_parent,has_global_ultimate,has_domestic_ultimate,has_ticker,has_registration_number,has_company_description,has_legal_status,has_ownership_type,has_entity_type,has_company_status,credibility_score_norm,missing_ratio_contact,missing_ratio_ownership,missing_ratio_financial,missing_ratio_it,missing_ratio_codes,missing_ratio_overall,log_org_complexity_count,employees_total_missing,log_employees_total,employees_single_site_missing,employees_single_site,revenue_usd_missing,log_revenue_usd,market_value_usd_missing,log_market_value_usd,corporate_family_members_missing,log_corporate_family_members,company_age_missing,log_company_age,employee_concentration,revenue_per_employee,market_value_per_employee,latitude_missing,latitude,longitude_missing,longitude,parent_foreign_flag,global_ultimate_foreign_flag,multinational_flag,num_countries_reported,pc_midpoint_missing,log_pc_midpoint,desktops_midpoint_missing,log_desktops_midpoint,laptops_midpoint_missing,log_laptops_midpoint,routers_midpoint_missing,log_routers_midpoint,servers_midpoint_missing,log_servers_midpoint,storage_devices_midpoint_missing,log_storage_devices_midpoint,log_it_assets_total,it_budget_missing,log_it_budget,it_spend_missing,log_it_spend,it_spend_rate,log_abs_it_budget_gap,it_assets_per_employee,it_spend_per_employee,sic_code_count,has_sic_code,sic8_code_count,has_sic8_code,naics_code_count,has_naics_code,nace2_code_count,has_nace2_code,anzsic_code_count,has_anzsic_code,isic4_code_count,has_isic4_code,registration_number_count,is_headquarters,is_headquarters_missing,is_domestic_ultimate,is_domestic_ultimate_missing,Region__Asia,Region__<NA>,Entity Type__Branch,Entity Type__Parent,Entity Type__Subsidiary,Entity Type__<NA>,Ownership Type__Non-Corporates,Ownership Type__Nonprofit,Ownership Type__Partnership,Ownership Type__Private,Ownership Type__Public Sector,Ownership Type__<NA>,Legal Status__0.0,Legal Status__100.0,Legal Status__101.0,Legal Status__12.0,Legal Status__120.0,Legal Status__13.0,Legal Status__3.0,Legal Status__50.0,Legal Status__8.0,Legal Status__<NA>,Franchise Status__FALSE,Franchise Status__False,Franchise Status__NO,Franchise Status__No,Franchise Status__<NA>,Manufacturing Status__Yes,Manufacturing Status__<NA>,Registration Number Type__indonesia legalization number,Registration Number Type__<NA>,cluster
5,-0.280669,2.028029,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,-2.628755,0.0,0.0,0.677902,-0.027865,0.0,3.166961,1.102224,0.687262,1.436245,-0.391397,0.0,-1.044719,-0.015288,-0.227341,0.0,1.132891,0.0,1.221989,0.0,-0.781694,3.169207,-0.083412,-1.316343,-0.145532,-0.145752,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,-0.484601,-0.203701,-0.050213,-0.284045,-0.038634,1.262704,-0.177458,1.247272,-0.028089,1.221947,-0.11092,-0.395814,0.0,1.253754,0.0,1.272335,0.796239,1.292383,-0.511873,-0.152715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.797430,2.068834,-0.854277,0.0,-0.01081,-0.030587,-0.02861,-2.601305,-0.021623,2.628755,-0.337021,-0.01081,12.549900,-0.059308,-0.056254,-0.056254,-2.595900,-0.02861,-0.100153,-0.015288,-0.630749,-0.097745,-0.018725,-0.01081,0.64616,-0.177351,0.177351,-0.030587,0.030587,-1
6,-0.280669,-0.493089,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,-0.274678,-0.027865,0.0,-0.315760,-0.213986,0.687262,0.308084,-0.391397,0.0,2.138134,-0.015288,1.558321,0.0,0.981420,0.0,-1.057564,0.0,-0.781694,-0.315536,0.476611,0.759680,-0.110707,-0.145752,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,1.500908,-0.203701,-0.050213,3.520563,-0.038634,-0.791951,4.554761,-0.801750,-0.028089,-0.818366,-0.11092,1.718426,0.0,0.926784,0.0,0.931572,0.796229,0.936249,-0.462743,-0.132016,0.0,0.0,0.0,0.0,0.0,0.0


=== Cluster 0 (n=104) ===


,has_website,has_phone,has_address,has_city,has_state,has_postal_code,has_country,has_region,has_company_name,has_parent,has_global_ultimate,has_domestic_ultimate,has_ticker,has_registration_number,has_company_description,has_legal_status,has_ownership_type,has_entity_type,has_company_status,credibility_score_norm,missing_ratio_contact,missing_ratio_ownership,missing_ratio_financial,missing_ratio_it,missing_ratio_codes,missing_ratio_overall,log_org_complexity_count,employees_total_missing,log_employees_total,employees_single_site_missing,employees_single_site,revenue_usd_missing,log_revenue_usd,market_value_usd_missing,log_market_value_usd,corporate_family_members_missing,log_corporate_family_members,company_age_missing,log_company_age,employee_concentration,revenue_per_employee,market_value_per_employee,latitude_missing,latitude,longitude_missing,longitude,parent_foreign_flag,global_ultimate_foreign_flag,multinational_flag,num_countries_reported,pc_midpoint_missing,log_pc_midpoint,desktops_midpoint_missing,log_desktops_midpoint,laptops_midpoint_missing,log_laptops_midpoint,routers_midpoint_missing,log_routers_midpoint,servers_midpoint_missing,log_servers_midpoint,storage_devices_midpoint_missing,log_storage_devices_midpoint,log_it_assets_total,it_budget_missing,log_it_budget,it_spend_missing,log_it_spend,it_spend_rate,log_abs_it_budget_gap,it_assets_per_employee,it_spend_per_employee,sic_code_count,has_sic_code,sic8_code_count,has_sic8_code,naics_code_count,has_naics_code,nace2_code_count,has_nace2_code,anzsic_code_count,has_anzsic_code,isic4_code_count,has_isic4_code,registration_number_count,is_headquarters,is_headquarters_missing,is_domestic_ultimate,is_domestic_ultimate_missing,Region__Asia,Region__<NA>,Entity Type__Branch,Entity Type__Parent,Entity Type__Subsidiary,Entity Type__<NA>,Ownership Type__Non-Corporates,Ownership Type__Nonprofit,Ownership Type__Partnership,Ownership Type__Private,Ownership Type__Public Sector,Ownership Type__<NA>,Legal Status__0.0,Legal Status__100.0,Legal Status__101.0,Legal Status__12.0,Legal Status__120.0,Legal Status__13.0,Legal Status__3.0,Legal Status__50.0,Legal Status__8.0,Legal Status__<NA>,Franchise Status__FALSE,Franchise Status__False,Franchise Status__NO,Franchise Status__No,Franchise Status__<NA>,Manufacturing Status__Yes,Manufacturing Status__<NA>,Registration Number Type__indonesia legalization number,Registration Number Type__<NA>,cluster
99,-0.280669,-0.493089,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,-1.523583,-0.750715,0.015288,-2.628755,0.0,0.0,-1.227258,-0.027865,0.0,3.166961,-0.213986,0.687262,0.684137,-0.391397,0.0,-1.044719,-0.015288,-0.227341,0.0,-1.235520,0.0,-1.057564,0.0,0.557253,3.169207,-0.083412,-1.316343,-0.145532,-0.145752,0.535968,-0.015032,0.536329,-0.011859,-0.044611,8.281628,8.281628,7.72719,-0.184243,-0.484601,-0.203701,-0.050213,-0.284045,-0.038634,1.262704,-0.177458,-0.801750,-0.028089,-0.818366,-0.11092,-0.395814,0.0,-1.222990,0.0,-1.219741,-1.256483,-1.216723,-0.511873,-0.152715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.254029,-0.483364,-0.854277,0.0,-0.01081,-0.030587,-0.02861,-2.601305,-0.021623,2.628755,2.967174,-0.01081,-0.079682,-0.059308,-0.056254,-0.056254,-2.595900,-0.02861,-0.100153,-0.015288,-0.630749,-0.097745,-0.018725,-0.01081,0.64616,-0.177351,0.177351,-0.030587,0.030587,0
377,-0.280669,2.028029,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,0.677902,-0.809810,0.0,-0.315760,-0.872091,0.687262,-0.444024,-0.391397,0.0,0.479959,-0.015288,-0.116991,0.0,0.681530,0.0,0.828478,0.0,-0.436112,-0.315536,1.182157,0.759680,-0.077132,-0.063620,0.535968,-0.015032,0.536329,-0.011859,-0.044611,8.281628,8.281628,7.72719,-0.184243,-0.484601,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.801750,-0.028089,-0.818366,-0.11092,-0.395814,0.0,0.741707,0.0,0.738687,0.796207,0.734668,-0.195783,-0.033657,0.0,0.0,0.0,0.0,


=== Cluster 1 (n=65) ===


...

,has_website,has_phone,has_address,has_city,has_state,has_postal_code,has_country,has_region,has_company_name,has_parent,has_global_ultimate,has_domestic_ultimate,has_ticker,has_registration_number,has_company_description,has_legal_status,has_ownership_type,has_entity_type,has_company_status,credibility_score_norm,missing_ratio_contact,missing_ratio_ownership,missing_ratio_financial,missing_ratio_it,missing_ratio_codes,missing_ratio_overall,log_org_complexity_count,employees_total_missing,log_employees_total,employees_single_site_missing,employees_single_site,revenue_usd_missing,log_revenue_usd,market_value_usd_missing,log_market_value_usd,corporate_family_members_missing,log_corporate_family_members,company_age_missing,log_company_age,employee_concentration,revenue_per_employee,market_value_per_employee,latitude_missing,latitude,longitude_missing,longitude,parent_foreign_flag,global_ultimate_foreign_flag,multinational_flag,num_countries_reported,pc_midpoint_missing,log_pc_midpoint,desktops_midpoint_missing,log_desktops_midpoint,laptops_midpoint_missing,log_laptops_midpoint,routers_midpoint_missing,log_routers_midpoint,servers_midpoint_missing,log_servers_midpoint,storage_devices_midpoint_missing,log_storage_devices_midpoint,log_it_assets_total,it_budget_missing,log_it_budget,it_spend_missing,log_it_spend,it_spend_rate,log_abs_it_budget_gap,it_assets_per_employee,it_spend_per_employee,sic_code_count,has_sic_code,sic8_code_count,has_sic8_code,naics_code_count,has_naics_code,nace2_code_count,has_nace2_code,anzsic_code_count,has_anzsic_code,isic4_code_count,has_isic4_code,registration_number_count,is_headquarters,is_headquarters_missing,is_domestic_ultimate,is_domestic_ultimate_missing,Region__Asia,Region__<NA>,Entity Type__Branch,Entity Type__Parent,Entity Type__Subsidiary,Entity Type__<NA>,Ownership Type__Non-Corporates,Ownership Type__Nonprofit,Ownership Type__Partnership,Ownership Type__Private,Ownership Type__Public Sector,Ownership Type__<NA>,Legal Status__0.0,Legal Status__100.0,Legal Status__101.0,Legal Status__12.0,Legal Status__120.0,Legal Status__13.0,Legal Status__3.0,Legal Status__50.0,Legal Status__8.0,Legal Status__<NA>,Franchise Status__FALSE,Franchise Status__False,Franchise Status__NO,Franchise Status__No,Franchise Status__<NA>,Manufacturing Status__Yes,Manufacturing Status__<NA>,Registration Number Type__indonesia legalization number,Registration Number Type__<NA>,cluster
1,-0.280669,2.028029,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,0.677902,-0.80981,0.0,-0.31576,-0.872091,0.687262,-0.444024,-0.391397,0.0,0.823325,-0.015288,-0.026705,0.0,0.563102,0.0,-1.057564,0.0,-0.781694,-0.315536,1.111311,0.75968,-0.129174,-0.145752,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,-0.484601,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.80175,-0.028089,-0.818366,-0.11092,-0.395814,0.0,0.321387,0.0,0.300624,0.795874,0.276956,-0.338023,-0.146465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.79743,-0.483364,1.17058,0.0,-0.01081,-0.030587,-0.02861,0.384422,-0.021623,-0.380408,-0.337021,-0.01081,-0.079682,-0.059308,-0.056254,-0.056254,0.385223,-0.02861,-0.100153,-0.015288,-0.630749,-0.097745,-0.018725,-0.01081,0.64616,-0.177351,0.177351,-0.030587,0.030587,43
36,-0.280669,2.028029,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,0.677902,-0.80981,0.0,-0.31576,-0.872091,0.687262,-0.444024,-0.391397,0.0,0.823325,-0.015288,-0.026705,0.0,0.920271,0.0,1.083640,0.0,-0.436112,-0.315536,0.587165,0.75968,0.056090,0.186188,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,1.500908,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.80175,-0.028089,-0.818366,-0.11092,0.892607,0.0,1.039914,0.0,1.049475,0.796234,1.059469,-0.206319,0.193948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


=== Cluster 44 (n=364) ===


,has_website,has_phone,has_address,has_city,has_state,has_postal_code,has_country,has_region,has_company_name,has_parent,has_global_ultimate,has_domestic_ultimate,has_ticker,has_registration_number,has_company_description,has_legal_status,has_ownership_type,has_entity_type,has_company_status,credibility_score_norm,missing_ratio_contact,missing_ratio_ownership,missing_ratio_financial,missing_ratio_it,missing_ratio_codes,missing_ratio_overall,log_org_complexity_count,employees_total_missing,log_employees_total,employees_single_site_missing,employees_single_site,revenue_usd_missing,log_revenue_usd,market_value_usd_missing,log_market_value_usd,corporate_family_members_missing,log_corporate_family_members,company_age_missing,log_company_age,employee_concentration,revenue_per_employee,market_value_per_employee,latitude_missing,latitude,longitude_missing,longitude,parent_foreign_flag,global_ultimate_foreign_flag,multinational_flag,num_countries_reported,pc_midpoint_missing,log_pc_midpoint,desktops_midpoint_missing,log_desktops_midpoint,laptops_midpoint_missing,log_laptops_midpoint,routers_midpoint_missing,log_routers_midpoint,servers_midpoint_missing,log_servers_midpoint,storage_devices_midpoint_missing,log_storage_devices_midpoint,log_it_assets_total,it_budget_missing,log_it_budget,it_spend_missing,log_it_spend,it_spend_rate,log_abs_it_budget_gap,it_assets_per_employee,it_spend_per_employee,sic_code_count,has_sic_code,sic8_code_count,has_sic8_code,naics_code_count,has_naics_code,nace2_code_count,has_nace2_code,anzsic_code_count,has_anzsic_code,isic4_code_count,has_isic4_code,registration_number_count,is_headquarters,is_headquarters_missing,is_domestic_ultimate,is_domestic_ultimate_missing,Region__Asia,Region__<NA>,Entity Type__Branch,Entity Type__Parent,Entity Type__Subsidiary,Entity Type__<NA>,Ownership Type__Non-Corporates,Ownership Type__Nonprofit,Ownership Type__Partnership,Ownership Type__Private,Ownership Type__Public Sector,Ownership Type__<NA>,Legal Status__0.0,Legal Status__100.0,Legal Status__101.0,Legal Status__12.0,Legal Status__120.0,Legal Status__13.0,Legal Status__3.0,Legal Status__50.0,Legal Status__8.0,Legal Status__<NA>,Franchise Status__FALSE,Franchise Status__False,Franchise Status__NO,Franchise Status__No,Franchise Status__<NA>,Manufacturing Status__Yes,Manufacturing Status__<NA>,Registration Number Type__indonesia legalization number,Registration Number Type__<NA>,cluster
20,-0.280669,-0.493089,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,-0.274678,-0.027865,0.0,-0.31576,-0.872091,0.687262,-0.06797,0.234799,0.0,1.118969,-0.015288,0.103709,0.0,1.171969,0.0,1.267509,0.0,1.097376,-0.315536,0.476611,0.75968,0.571834,0.700981,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,1.500908,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.80175,-0.028089,-0.818366,-0.11092,0.892607,0.0,1.087491,0.0,1.099059,0.796239,1.11129,-0.326689,0.121379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.79743,-0.483364,1.17058,0.0,-0.01081,-0.030587,-0.02861,0.384422,-0.021623,-0.380408,-0.337021,-0.01081,-0.079682,-0.059308,-0.056254,-0.056254,0.385223,-0.02861,-0.100153,-0.015288,-0.630749,-0.097745,-0.018725,-0.01081,0.64616,-0.177351,0.177351,-0.030587,0.030587,44
24,-0.280669,-0.493089,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,-0.274678,-0.027865,0.0,-0.31576,-0.872091,0.687262,-0.06797,-0.391397,0.0,0.954369,-0.015288,0.023454,0.0,0.774845,0.0,0.952808,0.0,-0.638264,-0.315536,-1.040779,0.75968,-0.087523,-0.050248,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,1.500908,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.80175,-0.028089,-0.818366,-0.11092,0.892607,0.0,0.856926,0.0,0.858768,0.796233,0.86016,-0.267430,-0.052975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.


=== Cluster 45 (n=451) ===


,has_website,has_phone,has_address,has_city,has_state,has_postal_code,has_country,has_region,has_company_name,has_parent,has_global_ultimate,has_domestic_ultimate,has_ticker,has_registration_number,has_company_description,has_legal_status,has_ownership_type,has_entity_type,has_company_status,credibility_score_norm,missing_ratio_contact,missing_ratio_ownership,missing_ratio_financial,missing_ratio_it,missing_ratio_codes,missing_ratio_overall,log_org_complexity_count,employees_total_missing,log_employees_total,employees_single_site_missing,employees_single_site,revenue_usd_missing,log_revenue_usd,market_value_usd_missing,log_market_value_usd,corporate_family_members_missing,log_corporate_family_members,company_age_missing,log_company_age,employee_concentration,revenue_per_employee,market_value_per_employee,latitude_missing,latitude,longitude_missing,longitude,parent_foreign_flag,global_ultimate_foreign_flag,multinational_flag,num_countries_reported,pc_midpoint_missing,log_pc_midpoint,desktops_midpoint_missing,log_desktops_midpoint,laptops_midpoint_missing,log_laptops_midpoint,routers_midpoint_missing,log_routers_midpoint,servers_midpoint_missing,log_servers_midpoint,storage_devices_midpoint_missing,log_storage_devices_midpoint,log_it_assets_total,it_budget_missing,log_it_budget,it_spend_missing,log_it_spend,it_spend_rate,log_abs_it_budget_gap,it_assets_per_employee,it_spend_per_employee,sic_code_count,has_sic_code,sic8_code_count,has_sic8_code,naics_code_count,has_naics_code,nace2_code_count,has_nace2_code,anzsic_code_count,has_anzsic_code,isic4_code_count,has_isic4_code,registration_number_count,is_headquarters,is_headquarters_missing,is_domestic_ultimate,is_domestic_ultimate_missing,Region__Asia,Region__<NA>,Entity Type__Branch,Entity Type__Parent,Entity Type__Subsidiary,Entity Type__<NA>,Ownership Type__Non-Corporates,Ownership Type__Nonprofit,Ownership Type__Partnership,Ownership Type__Private,Ownership Type__Public Sector,Ownership Type__<NA>,Legal Status__0.0,Legal Status__100.0,Legal Status__101.0,Legal Status__12.0,Legal Status__120.0,Legal Status__13.0,Legal Status__3.0,Legal Status__50.0,Legal Status__8.0,Legal Status__<NA>,Franchise Status__FALSE,Franchise Status__False,Franchise Status__NO,Franchise Status__No,Franchise Status__<NA>,Manufacturing Status__Yes,Manufacturing Status__<NA>,Registration Number Type__indonesia legalization number,Registration Number Type__<NA>,cluster
21,-0.280669,-0.493089,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,-0.274678,-0.027865,0.0,-0.31576,-0.872091,0.687262,-0.06797,-0.391397,0.0,0.728742,-0.015288,-0.056800,0.0,0.692064,0.0,0.826196,0.0,-0.292682,-0.315536,0.587165,0.75968,-0.097870,-0.093547,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,-0.484601,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.80175,-0.028089,-0.818366,-0.11092,-0.395814,0.0,0.617339,0.0,0.609077,0.796232,0.599208,-0.307344,-0.114269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.254029,-0.483364,-0.854277,0.0,-0.01081,-0.030587,-0.02861,0.384422,-0.021623,-0.380408,-0.337021,-0.01081,-0.079682,-0.059308,-0.056254,-0.056254,0.385223,-0.02861,-0.100153,-0.015288,-0.630749,-0.097745,-0.018725,-0.01081,0.64616,-0.177351,0.177351,-0.030587,0.030587,45
68,-0.280669,-0.493089,0.204323,0.252766,0.253027,0.248289,0.0,0.0,0.0,0.0,0.0,0.0,-0.021623,0.656348,-0.750715,0.015288,0.380408,0.0,0.0,-0.274678,-0.027865,0.0,-0.31576,-0.872091,0.687262,-0.06797,-0.391397,0.0,0.954369,-0.015288,0.023454,0.0,0.958324,0.0,1.024896,0.0,-0.781694,-0.315536,-1.040779,0.75968,0.065254,0.022026,0.535968,-0.015032,0.536329,-0.011859,-0.044611,-0.120749,-0.120749,-0.118245,-0.184243,-0.484601,-0.203701,-0.050213,-0.284045,-0.038634,-0.791951,-0.177458,-0.80175,-0.028089,-0.818366,-0.11092,-0.395814,0.0,0.910068,0.0,0.914152,0.796233,0.918042,-0.372793,-0.018485,0.0,0.0,0.0,0.0,0.0,0.

### HDBSCAN identified **46** clusters among 8,559 companies, with approximately **24.19%** of firms classified as **noise**. Noise firms exhibit sparse or inconsistent records and are treated as high-uncertainty cases rather than forced into clusters.

In [38]:
pd.Series(labels).value_counts().drop(-1).head(5)

45    451
44    364
27    340
34    336
19    276
Name: count, dtype: int64

## Interpretation:

- These are the 5 largest real clusters
- They cover a substantial portion of the dataset



### Cluster 45 – Large, IT-intensive subsidiary enterprises

| Key Signal | Direction | Meaning |
|-----------|----------|--------|
| `Entity_Type__Subsidiary` | High | Firms are predominantly subsidiaries, indicating layered organisational structures |
| `log_revenue_usd` | High | High operating revenue and large-scale business activity |
| `log_market_value_usd` | High | Strong firm valuation and economic significance |
| `log_it_budget` | High | Substantial allocation of resources to IT infrastructure |
| `it_spend_rate` | High | IT spending scales aggressively with firm size |
| `log_it_spend` | High | High absolute IT expenditure |
| `log_abs_it_budget_gap` | High | Significant IT budget adjustments, suggesting active IT planning or expansion |
| `log_employees_total` | High | Large workforce and organisational scale |
| `missing_ratio_it` | Low | IT-related data is largely complete and well-reported |
| `servers_midpoint_missing` | Low | Consistent reporting of core IT infrastructure assets |
| `Entity_Type__Branch` | Low | Unlikely to operate as small or peripheral branch entities |

Cluster 45 captures large, economically significant subsidiary enterprises with complex organisational structures and strong reliance on IT infrastructure. These firms exhibit high revenue, valuation, headcount, and IT spending, reflecting operations at substantial scale. The low level of missingness across IT and infrastructure-related variables indicates mature internal reporting practices and well-documented systems. Risk exposure in this cluster is therefore driven primarily by **operational scale and governance complexity**, rather than data opacity or lack of transparency.

In [39]:
cid_1 = 45
cluster_profiles_no_noise.loc[cid_1].sort_values(ascending=False).head(10)


Entity Type__Subsidiary    1.166090
log_revenue_usd            0.833412
log_market_value_usd       0.831414
log_it_budget              0.796774
it_spend_rate              0.796159
log_it_spend               0.796075
log_abs_it_budget_gap      0.794662
employee_concentration     0.759680
missing_ratio_codes        0.687262
log_employees_total        0.679370
Name: 45, dtype: float64

In [40]:
cluster_profiles_no_noise.loc[cid_1].sort_values(ascending=True).head(10)

missing_ratio_it                   -0.872091
storage_devices_midpoint_missing   -0.818366
servers_midpoint_missing           -0.801750
Entity Type__Branch                -0.792881
routers_midpoint_missing           -0.791951
has_company_description            -0.750715
Franchise Status__FALSE            -0.630749
log_company_age                    -0.514108
has_phone                          -0.493089
log_pc_midpoint                    -0.484601
Name: 45, dtype: float64

### Cluster 44 - High-infrastructure, IT-heavy subsidiaries

| Key Signal | Direction | Meaning |
|-----------|----------|--------|
| `log_pc_midpoint` | High | Very high density of endpoint devices across the organisation |
| `log_employees_total` | High | Large overall workforce size |
| `log_it_assets_total` | High | Extensive IT infrastructure and asset base |
| `log_market_value_usd` | High | Strong firm valuation and economic significance |
| `log_it_budget` | High | Substantial allocation of resources to IT operations |
| `log_it_spend` | High | High absolute expenditure on IT systems |
| `it_spend_rate` | High | IT spending scales aggressively with organisational size |
| `log_abs_it_budget_gap` | High | Active adjustment and expansion of IT budgets |
| `Entity Type_Subsidiary` | High | Predominantly subsidiary entities within larger corporate groups |
| `missing_ratio_it`| Low | Strong completeness and reliability of IT-related data |
| `servers_midpoint_missing` | Low | Consistent reporting of server infrastructure |
| `storage_devices_midpoint_missing` | Low | Consistent reporting of storage infrastructure |
| `routers_midpoint_missing` | Low | Consistent reporting of network infrastructure |
| `Entity Type_Branch` | Low | Less likely to operate as small or peripheral branch entities |

Cluster 44 represents large subsidiary organisations characterised by exceptionally high endpoint density and extensive IT infrastructure. Elevated workforce size, market valuation, and IT expenditure indicate enterprises operating at substantial scale with strong reliance on internal digital systems to support daily operations. The presence of significant IT budget gaps suggests active investment, expansion, or ongoing digital transformation rather than static IT environments. Low levels of IT and infrastructure data missingness reflect **mature governance and well-established reporting practices**. Overall, risk exposure within this cluster is driven primarily by **infrastructure scale, asset sprawl, and change-management complexity**, rather than **data opacity or informational gaps**.

In [41]:
cid_2 = 44
cluster_profiles_no_noise.loc[cid_2].sort_values(ascending=False).head(10)

log_pc_midpoint            1.540776
Entity Type__Subsidiary    1.170580
log_employees_total        1.103940
log_it_assets_total        0.939931
log_market_value_usd       0.915978
log_abs_it_budget_gap      0.876260
log_it_spend               0.874172
log_it_budget              0.871707
log_revenue_usd            0.823631
it_spend_rate              0.796227
Name: 44, dtype: float64

In [42]:
cluster_profiles_no_noise.loc[cid_2].sort_values(ascending=True).head(10)

missing_ratio_it                   -0.866667
servers_midpoint_missing           -0.801750
storage_devices_midpoint_missing   -0.801550
Entity Type__Branch                -0.797430
routers_midpoint_missing           -0.791951
has_company_description            -0.750715
Franchise Status__FALSE            -0.630749
log_company_age                    -0.589069
has_phone                          -0.493089
Entity Type__Parent                -0.483364
Name: 44, dtype: float64

### Cluster 27 – Data-poor, low-IT branch organisations

| Key Signal | Direction | Meaning |
|-----------|----------|--------|
| `routers_midpoint_missing` | High | Network infrastructure details are largely unreported |
| `servers_midpoint_missing` | High | Server infrastructure information is frequently missing |
| `storage_devices_midpoint_missing` | High | Storage infrastructure is poorly documented |
| `missing_ratio_it` | High | IT-related data is largely incomplete |
| `missing_ratio_overall` | High | High overall data missingness across features |
| `missing_ratio_codes` | High | Operational or classification codes are often absent |
| `Entity Type_Branch` | High | Firms are predominantly small branch-level entities |
| `Franchise Status__NA` | High | Franchise status is frequently unreported or unclear |
| `longitude_missing` | High | Geographic location data is incomplete |
| `latitude_missing` | High | Geographic location data is incomplete |
| `employee_concentration` | Low | Workforce is not concentrated at large operating sites |
| `log_employees_total` | Low | Small overall workforce size |
| `log_revenue_usd` | Low | Low operating revenue |
| `log_market_value_usd` | Low | Low firm valuation |
| `log_it_budget` | Low | Limited allocation of resources to IT |
| `log_it_spend` | Low | Low absolute IT expenditure |
| `it_spend_rate` | Low | IT spending does not scale with operations |
| `log_abs_it_budget_gap` | Low | Minimal IT budget adjustment or investment activity |
| `Entity Type_Subsidiary` | Low | Unlikely to operate as subsidiary entities |

Cluster 27 represents small, branch-level organisations characterised by extensive data incompleteness and limited IT investment. High missingness across IT infrastructure, operational, and geographic variables suggests weak internal reporting practices and low system maturity. These firms operate at relatively small scale, with low revenue, valuation, workforce size, and minimal IT budgets, indicating limited reliance on digital infrastructure. Risk exposure in this cluster is driven primarily by **data opacity and informational gaps** rather than **operational complexity**, making these entities difficult to assess due to lack of visibility rather than scale or technological sophistication.

In [43]:
cid_3 = 27
cluster_profiles_no_noise.loc[cid_3].sort_values(ascending=False).head(10)

routers_midpoint_missing            1.262704
Entity Type__Branch                 1.254029
servers_midpoint_missing            1.247272
storage_devices_midpoint_missing    1.221947
missing_ratio_it                    1.102224
missing_ratio_overall               1.080100
missing_ratio_codes                 0.687262
Franchise Status__<NA>              0.646160
longitude_missing                   0.536329
latitude_missing                    0.535968
Name: 27, dtype: float64

In [44]:
cluster_profiles_no_noise.loc[cid_3].sort_values(ascending=True).head(10)

employee_concentration    -1.316343
it_spend_rate             -1.244418
log_revenue_usd           -1.226953
log_it_budget             -1.215774
log_it_spend              -1.212738
log_abs_it_budget_gap     -1.209949
log_market_value_usd      -1.048938
log_employees_total       -1.044719
Entity Type__Subsidiary   -0.854277
has_company_description   -0.750715
Name: 27, dtype: float64

### Cluster 34 - Mature, high-credibility and transparent firms

| Key Signal | Direction | Meaning |
|-----------|----------|--------|
| `has_company_description` | High | Public-facing company information is consistently available |
| `credibility_score_norm` | High | High overall credibility and trustworthiness score |
| `log_company_age` | High | Older, more established organisations |
| `it_spend_rate` | High | IT spending scales reliably with operations |
| `employee_concentration` | High | Workforce is concentrated at main operating sites |
| `log_revenue_usd` | High | Stable and relatively high operating revenue |
| `has_phone` | High | Contact information is consistently available |
| `log_it_budget` | High | Sustained allocation of resources to IT |
| `log_it_spend` | High | Consistent IT expenditure |
| `log_market_value_usd` | High | Solid firm valuation and economic standing |
| `latitude_missing` | Low | Geographic location data is consistently available |
| `longitude_missing` | Low | Geographic location data is consistently available |
| `missing_ratio_overall` | Low | Low overall data missingness |
| `missing_ratio_it` | Low | IT-related data is largely complete |
| `servers_midpoint_missing` | Low | Server infrastructure is well-documented |
| `storage_devices_midpoint_missing` | Low | Storage infrastructure is well-documented |
| `routers_midpoint_missing` | Low | Network infrastructure is well-documented |
| `Entity Type_Branch` | Low | Less likely to operate as small branch entities |
| `missing_ratio_codes` | Low | Operational and classification codes are consistently reported |

Cluster 34 represents mature, well-established organisations characterised by high credibility and strong data transparency. These firms exhibit consistent availability of public-facing information, reliable contact details, and comprehensive reporting across operational, geographic, and IT-related dimensions. Elevated company age, stable revenue, and sustained IT spending suggest organisations with settled operating models rather than rapid expansion or restructuring. Low levels of missingness across infrastructure and descriptive variables indicate strong governance and disciplined internal processes. Overall, risk exposure for this cluster is comparatively low and driven more by **standard operational considerations** than by **data opacity, infrastructure sprawl, or governance complexity**.

In [45]:
cid_4 = 34
cluster_profiles.loc[cid_4].sort_values(ascending=False).head(10)

has_company_description    1.232883
credibility_score_norm     1.083316
log_company_age            1.036860
it_spend_rate              0.795913
employee_concentration     0.759680
log_revenue_usd            0.733079
has_phone                  0.722450
log_it_budget              0.708149
log_it_spend               0.703706
log_market_value_usd       0.702375
Name: 34, dtype: float64

In [46]:
cluster_profiles.loc[cid_4].sort_values(ascending=True).head(10)

latitude_missing                   -1.865785
longitude_missing                  -1.864528
missing_ratio_overall              -0.995793
missing_ratio_it                   -0.860339
storage_devices_midpoint_missing   -0.812294
servers_midpoint_missing           -0.801750
routers_midpoint_missing           -0.791951
Entity Type__Branch                -0.681425
missing_ratio_codes                -0.402485
missing_ratio_contact              -0.395565
Name: 34, dtype: float64

### Cluster 16 - Small branch firms with limited IT visibility despite formal registration

| Key Signal | Direction | Meaning |
|-----------|----------|--------|
| `Franchise Status_FALSE` | High | Firms are predominantly non-franchise entities |
| `has_company_description` | High | Public-facing company information is available |
| `routers_midpoint_missing` | High | Network infrastructure details are largely unreported |
| `servers_midpoint_missing` | High | Server infrastructure information is frequently missing |
| `storage_devices_midpoint_missing` | High | Storage infrastructure is poorly documented |
| `missing_ratio_it` | High | IT-related data is largely incomplete |
| `Entity Type_Branch` | High | Firms are predominantly branch-level entities |
| `credibility_score_norm` | High | Relatively high credibility despite operational simplicity |
| `has_registration_number` | High | Formal registration information is available |
| `log_corporate_family_members` | High | Firms belong to small corporate groups |
| `employee_concentration` | Low | Workforce is not concentrated at large operating sites |
| `log_employees_total` | Low | Small overall workforce size |
| `log_revenue_usd` | Low | Low operating revenue |
| `log_market_value_usd` | Low | Low firm valuation |
| `log_it_budget` | Low | Limited allocation of resources to IT |
| `log_it_spend` | Low | Low absolute IT expenditure |
| `it_spend_rate` | Low | IT spending does not scale with operations |
| `log_abs_it_budget_gap` | Low | Minimal IT budget adjustment or expansion activity |

Cluster 16 represents small, branch-level organisations with limited operational scale and minimal reliance on IT infrastructure. While these firms exhibit relatively strong formal indicators such as company descriptions, registration numbers, and moderate credibility scores, they display substantial gaps in IT and infrastructure reporting. Low workforce size, revenue, valuation, and IT spending suggest simple operating models with limited technological dependence. Risk exposure in this cluster is driven primarily by **weak IT visibility and infrastructure transparency** rather than **governance or organisational complexity**, making oversight challenging due to incomplete technical information despite basic corporate legitimacy.

In [47]:
cid_5 = 16
cluster_profiles.loc[cid_5].sort_values(ascending=False).head(10)

it_assets_per_employee     2.965122
Franchise Status__FALSE    1.585417
Entity Type__Branch        1.151456
has_company_description    0.915507
employee_concentration     0.759680
longitude_missing          0.536329
latitude_missing           0.535968
Legal Status__3.0          0.385223
Ownership Type__Private    0.384422
has_ownership_type         0.380408
Name: 16, dtype: float64

In [48]:
cluster_profiles.loc[cid_5].sort_values(ascending=True).head(10)

missing_ratio_codes      -2.016983
missing_ratio_overall    -1.948239
Franchise Status__<NA>   -1.547605
it_spend_rate            -1.256483
log_revenue_usd          -1.235520
log_it_budget            -1.222990
log_it_spend             -1.219741
log_abs_it_budget_gap    -1.216723
log_market_value_usd     -1.057564
log_company_age          -1.040779
Name: 16, dtype: float64

### Summarise

| Cluster | Dominant Risk Type | Key Driver |
|--------|------------------|-----------|
| 27 | Transparency & data quality risk | High missingness across operational and IT variables |
| 16 | IT visibility risk | Poor infrastructure reporting despite formal registration |
| 34 | Low risk (high transparency) | Mature operations with strong disclosure quality |
| 44 | Operational complexity risk | Dense IT assets and infrastructure sprawl |
| 45 | Governance & coordination risk | Large-scale subsidiary structures with heavy IT reliance |